In [57]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
import numpy as np
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import *
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score
import plotly.express as px
import pickle

In [58]:
def buildModel(overUnderThreshold: int, data: pd.DataFrame, file: str):
    overUnder = []
    for row in data.itertuples():
        if row.fantasy_points_per_game_next_year >= overUnderThreshold:
            overUnder.append(1)
        else:
            overUnder.append(0)
    data['overUnder'] = overUnder
    remove = ['espn_id', 'season', 'fantasy_points_per_game_next_year', 'fantasy_points_next_year', 'qualified_n-1', 'qualified_n-2', 'qualified', 'overUnder']
    y = data['overUnder']
    X = data.drop(labels=remove, axis=1)
    highestMean = 0

    bestDepth = 0
    bestEst = 0
    bestRate = 0
    # define model evaluation method
    cv = ms.StratifiedKFold(n_splits=10)
    for depth in range(2, 11, 2):
        for est in range(50, 251, 50):
            for rate in [0.1, 0.2, 0.3]:
                # fit model no training data
                model = xgb.XGBClassifier(max_depth=depth, n_estimators=est, learning_rate=rate)
                # evaluate model
                scores = ms.cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
                if scores.mean() > highestMean:
                    highestMean = scores.mean()
                    bestDepth = depth
                    bestEst = est
                    bestRate = rate
    optimalModel = xgb.XGBClassifier(max_depth=bestDepth, n_estimators=bestEst, learning_rate=bestRate)
    optimalModel.fit(X, y)
    pickle.dump(optimalModel, open(file, "wb"))
    return highestMean


In [59]:
df = pd.read_parquet("qb_data_complete.parquet")

In [60]:
df = df.dropna(subset=['fantasy_points_next_year'])

In [66]:
df = df[df.qualified == 1].copy()

In [67]:
px.histogram(df, x="fantasy_points_per_game_next_year", nbins=20)

In [68]:
thresholdAndNames = {"OverUnderFifteen.pkl": 15,
                     "OverUnderEighteen.pkl": 18, 
                     "OverUnderTwentyOne.pkl": 21,
                     "OverUnderTwentyFour.pkl": 24}

In [69]:
for key in thresholdAndNames.keys():
    mean = buildModel(thresholdAndNames[key], df, key)
    print(key, ":", mean)

OverUnderFifteen.pkl : 0.6973949579831934
OverUnderEighteen.pkl : 0.7845378151260504
OverUnderTwentyOne.pkl : 0.8811764705882353


/Users/benabraham/opt/anaconda3/envs/FantFootballV2/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 9 members, which is less than n_splits=10.

/Users/benabraham/opt/anaconda3/envs/FantFootballV2/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 9 members, which is less than n_splits=10.

/Users/benabraham/opt/anaconda3/envs/FantFootballV2/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 9 members, which is less than n_splits=10.

/Users/benabraham/opt/anaconda3/envs/FantFootballV2/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 9 members, which is less than n_splits=10.

/Users/benabraham/opt/anaconda3/envs/FantFootballV2/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning:


OverUnderTwentyFour.pkl : 0.9738655462184875
